# Analyse von Gesundheitsdaten 
## Projektarbeit Gruppe 2

### Konsolidierung aller benötigten Anforderungen innerhalb unseres Use Cases.

Projektteam: Sebastian Grab, Josua Klöble, Imanuel Ricker

Unser Datensatz ist auffindbar unter:

https://www.kaggle.com/datasets/sooyoungher/smoking-drinking-dataset



Unsere Dokumentation ist auffindbar unter:

https://grkconsulting.atlassian.net/l/cp/1v1GPp1w

Testnachricht für Josua

In [ ]:
# Import der benötigten Bibliotheken

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

# 1. Datentransformation & -exploration

## 1.1 Daten laden und verstehen

In diesem Abschnitt werden die Daten geladen und die Spaltenbezeichnungen geändert.

In [ ]:
# Lokaler Repository Pfad auslesen zum dynamischen Laden unabhängig des Nutzers:
path = str(os.getcwd()).replace("\Projektarbeit", "")

# Lesen der CSV Datei:
dataset = pd.read_csv(path + '\smoking_driking_dataset_Ver01.csv')

In [ ]:
# Umbenennen der Spalten:

dataset = dataset.rename(columns={'sex': 'Geschlecht', 'age': 'Alter', 'height': 'Größe', 'weight': 'Gewicht', 'waistline': 'Hüftumfang', 'sight_left': 'Sehkraft_links', 'sight_right': 'Sehkraft_rechts', 'hear_left': 'Hörkraft_links', 'hear_right': 'Hörkraft_rechts', 'SBP': 'Systolischer Blutdruck', 'DBP': 'Diastolischer Blutdruck', 'BLDS': 'Nüchterner Blutzucker', 'tot_chole': 'Totale Cholesterin', 'HDL_chole': 'HDL_Cholesterin', 'LDL_chole': 'LDL_Cholesterin', 'triglyceride': 'Triglycerid', 'hemoglobin': 'Hemoglobin', 'urine_protein': 'Urin_Proteine', 'serum_creatinine': 'Serum_Kreatinin', 'SGOT_AST': 'SGOT_AST', 'SGOT_ALT': 'SGOT_ALT', 'gamma_GTP': 'gamma_GTP', 'SMK_stat_type_cd': 'Raucher_Status', 'DRK_YN': 'Trinker'})

dataset.head()

In [ ]:
# Ausgabe der Spalten:

column_names = dataset.columns.values.tolist()
print(column_names)

Innerhalb unserer Dokumentation im Kapitel "Datensatz" findet sich eine übersichtliche Darstellung der jeweiligen Spalten. Ferner gehen wir dort auf deren Bedeutung und Auffälligkeiten bzw. Unstimmigkeiten in den Daten ein.

## 1.2 Daten transformieren

In diesem Abschnitt werden die Datentypen angepasst und neue Spalten hinzugefügt, sodass die weitere Arbeit mit den Daten vereinfacht wird.

In [ ]:
# Spalten transfomieren:

    # Trinker-Spalte: Y (für Yes) zu 1 & n (für No) zu 0

dataset["Trinker"] = dataset["Trinker"] == 'Y'
dataset["Trinker"] = dataset["Trinker"].astype(int)


    # Neue Spalten 'Mann' & 'Frau': 1 wenn true, 0 wenn false:

dataset['Mann'] = dataset['Geschlecht'] == 'Male'
dataset['Frau'] = dataset['Geschlecht'] == 'Female'

dataset['Mann'] = dataset['Mann'].astype(int)
dataset['Frau'] = dataset['Frau'].astype(int)

    # Entfernen der ursprünglichen 'Geschlecht'-Spalte:

dataset = dataset.drop('Geschlecht', axis=1)


    # Hörkraft-Spalten: 1 (für normal) bleibt 1 & 2 (für abnormal) zu 0

dataset["Hörkraft_links"] = dataset["Hörkraft_links"] == 1
dataset["Hörkraft_links"] = dataset["Hörkraft_links"].astype(int)

dataset["Hörkraft_rechts"] = dataset["Hörkraft_rechts"] == 1
dataset["Hörkraft_rechts"] = dataset["Hörkraft_rechts"].astype(int)


    # Hinzufügen einer berechneten Spalte - der Body-Mass-Index:

body_mass_index = dataset["Gewicht"].astype(np.float64) / ((dataset["Größe"].astype(np.float64) / 100) * (dataset["Größe"].astype(np.float64) / 100))

dataset["Body-Mass-Index"] = round(body_mass_index, 2)


    # Hinzufügen einer berechneten Spalte - der mittlere arterielle Blutdruck:

MAD = dataset["Diastolischer Blutdruck"] + (( dataset["Systolischer Blutdruck"] - dataset["Diastolischer Blutdruck"] ) * 0.5)

dataset["Mittlerer arterieller Blutdruck"] = MAD


dataset.head()

Neben der Normalisierung der Textspalten konnten wir hier zudem den Body-Mass-Index sowie den mittleren arteriellen Blutdruck berechnen.

## 1.3 Datenexploration

In diesem Abschnitt werden die Daten erneut, tiefergehend, betrachtet, sodass eine erweiterte Datenbereinigung ermöglicht wird. 

In [ ]:
# Datenexploration zur Erkennung von Bereinigungsbedarf:

    # Funktion zum Darstellen der Verteilung der Werte einer Spalte:

def bar_chart(column):
    share = (dataset[column].sort_values().value_counts(sort=False) / len(dataset)) * 100 
    plt.figure(figsize=(10, 6)) 
    share.plot(kind='bar') 
    plt.xlabel(share) 
    plt.ylabel('Prozentuale Häufigkeit')
    plt.grid(axis='y') 
    plt.show()


    # Funktion zum Darstellen der Korrelation der Werte mehrerer Spalten:

def corr(columns):
    plt.figure(figsize=(8, 8))
    data = dataset[columns]
    heatmap = sns.heatmap(data.corr(), annot=True, cmap="coolwarm")

In [ ]:
# Darstellen der Verteilung der jeweiligen Spalten:

column_names = dataset.columns.values.tolist()

for column in column_names:
    bar_chart(column)

Auffällige Datenpunkte:

1. Hüftumfang: Wenige Datenpunkte bei 999 cm
2. Sehkraft_links: Wenige Datenpunkte bei 9.9 (Verteilung der anderen Werte von 0,1 bis 2,0)
3. Sehkraft_rechts: Wenige Datenpunkte bei 9.9 (Verteilung der anderen Werte von 0,1 bis 2,0)

In [ ]:
corr(['Body-Mass-Index','Gewicht','Größe'])

Auffällige Zusammenhänge:

1. ...

# 2. Datenbereinigung

Auf Basis der Datenexploration lassen sich diese in diesem Abschnitt in ihrer Semantik bereinigen. 

In [ ]:
# Es gibt 57 Reihen, in denen ein Hüftumfang von 9,99 m angegeben wurde:

filtered_dataset = dataset[dataset['Hüftumfang'] == 999]
len(filtered_dataset.index)

    # Eliminierung der fehlerhaften Reihen:

dataset = dataset[dataset['Hüftumfang'] != 999]


# Es gibt mehrer Reihen, in denen die Sehkraft 9,9 angegeben wurde. Dies bedeutet, dass der Proband erblindet ist.

filtered_dataset = dataset[dataset['Sehkraft_rechts'] == 9.9]
len(filtered_dataset.index)

filtered_dataset = dataset[dataset['Sehkraft_links'] == 9.9]
len(filtered_dataset.index)


dataset.describe()

Kommentare

# 3. Feature Selektion & Engineering

Nun können wir die Features selektieren und engineeren.

## Blutdruck

Problem - Bedingungen um Blutdruck einzuteilen können nicht immer erfüllt werden. (Bspw. Sys. Blutdruck normal; Dias. Blutdruck optimal. Daher keine Einordung)

Lösungen:

    1. Tolernazen einbauen

    2. Nach mittleren Blutdruck gehen

    3. Nur Obergrenzen festlegen
    
    4. Sys. und Dias. Blutdruck eigenständig Einordungen. Anschließend beide Einordungen "vergleichen" und Mittlere Einordung nehemen. Eventuell mit Hang zum schlechteren.

Lösung 2:

BD_optimal --> <=119 / <=79 --> <100
BD_normal --> 120 - 129 / 80 - 84 --> >=100 - <106,5
BD_Grenzwert_normal --> 130 - 139 / 85 - 89 --> >=106,5 - <111,5
BD_Hypertonie Grad 1  --> 140 - 159 / 90 - 99 --> >=111,5 - <129
BD_Hypertonie Grad 2  --> 160 - 179 / 100 - 109 --> >=129 - <144
BD_Hypertonie Grad 3  --> >179 / >109 --> >=111,5 - >=144

In [ ]:
dataset["BD_optimal"] = dataset['Systolischer Blutdruck']
dataset["BD_normal"] = dataset['Systolischer Blutdruck']
dataset["BD_Grenzwert_normal"] = dataset['Systolischer Blutdruck']
dataset["BD_Hypertonie Grad 1"] = dataset['Systolischer Blutdruck']
dataset["BD_Hypertonie Grad 2"] = dataset['Systolischer Blutdruck']
dataset["BD_Hypertonie Grad 3"] = dataset['Systolischer Blutdruck']

In [ ]:
dataset["BD_optimal"] = (dataset['Mittlerer arterieller Blutdruck'] <100)
dataset["BD_optimal"] = dataset["BD_optimal"].astype(int)

dataset["BD_normal"] = (dataset['Mittlerer arterieller Blutdruck'] >= 100) & (dataset['Mittlerer arterieller Blutdruck'] < 106.5)
dataset["BD_normal"] = dataset["BD_normal"].astype(int) 

dataset["BD_Grenzwert_normal"] = (dataset['Mittlerer arterieller Blutdruck'] >= 106.5) & (dataset['Mittlerer arterieller Blutdruck'] < 111.5)
dataset["BD_Grenzwert_normal"] = dataset["BD_Grenzwert_normal"].astype(int) 

dataset["BD_Hypertonie Grad 1"] = (dataset['Mittlerer arterieller Blutdruck'] >= 111.5) & (dataset['Mittlerer arterieller Blutdruck'] < 129)
dataset["BD_Hypertonie Grad 1"] = dataset["BD_Hypertonie Grad 1"].astype(int) 

dataset["BD_Hypertonie Grad 2"] = (dataset['Mittlerer arterieller Blutdruck'] >= 129) & (dataset['Mittlerer arterieller Blutdruck'] < 144)
dataset["BD_Hypertonie Grad 2"] = dataset["BD_Hypertonie Grad 2"].astype(int) 

dataset["BD_Hypertonie Grad 3"] = (dataset['Mittlerer arterieller Blutdruck'] >=144)
dataset["BD_Hypertonie Grad 3"] = dataset["BD_Hypertonie Grad 3"].astype(int) 

In [ ]:
dummy = pd.get_dummies(dataset['Raucher_Status'], prefix='', prefix_sep='Raucher_')

# Verbinde Dummy-Variablen mit dem ursprünglichen DataFrame
dataset = pd.concat([dataset, dummy], axis=1)

# Entferne die ursprüngliche Spalte "Raucher_Status", wenn gewünscht
# dataset = dataset.drop('Raucher_Status', axis=1)

# Umbenennen der Dummy-Spalten
dataset = dataset.rename(columns={'Raucher_1.0': 'Raucher_nie', 'Raucher_2.0': 'Raucher_ehem', 'Raucher_3.0': 'Raucher_aktiv'})


In [ ]:

# Untersuchung der Korrelation aller Features:

plt.figure(figsize=(28, 28))
heatmap = sns.heatmap(dataset.corr(), annot=True, cmap="coolwarm")

plt.show()

Anmerkungen: 
